In [1]:
from main import generate_estimation
from math import ceil, floor
import time
import json
from datetime import datetime
import threading
import os
import numpy as np
# _, numOfCars = generate_estimation()
# print(_)
# numOfCars
# greenTime = min(numOfCars * 2 + 3, 120)
# print(numOfCars, greenTime)

In [2]:
# write code to remove the content of intersection_A.json, intersection_B.json, intersection_C.json, intersection_D.json
# after the simulation is done
#check if file is existed
if os.path.exists("intersection_A.json"):
    with open("intersection_A.json", "w") as file:
        file.write("")
if os.path.exists("intersection_B.json"):
    with open("intersection_B.json", "w") as file:
        file.write("")
if os.path.exists("intersection_C.json"):
    with open("intersection_C.json", "w") as file:
        file.write("")
if os.path.exists("intersection_D.json"):
    with open("intersection_D.json", "w") as file:
        file.write("")
if os.path.exists("merged_sorted_intersections.json"):
    with open("merged_sorted_intersections.json", "w") as file:
        file.write("")


In [3]:
class TrafficLight:
    def __init__(self, trafficID, lanes, generateCars=True):
        self.trafficID = trafficID
        self.lanes = lanes
        self.totalCars = 0
        self.carsPerLane = 0
        self.greenTime = 3
        self.generateCars = generateCars

    def update_traffic(self, totalCars):
        self.totalCars += totalCars
        self.carsPerLane = ceil(totalCars / self.lanes)
        # self.greenTime = min(self.carsPerLane * 2 + 3, 50)
        self.greenTime = max(min(ceil((self.totalCars * 2.5)/(self.lanes +1)), 50),3)
    def getTotalCars(self):
        return self.totalCars
    def getGreenTime(self):
        return self.greenTime
    def getTrafficID(self):
        return self.trafficID
    def resetGreenTime(self):
        self.greenTime = 3
    def resetTotalCars(self):
        if self.greenTime > 3:
            self.totalCars -= floor(max(0, (self.greenTime*(self.lanes + 1))//2.5))
            self.totalCars = max(0, self.totalCars)
        else:
            self.totalCars = 0
    def __str__(self):
        return (f"Traffic Light {self.trafficID} -> Lanes: {self.lanes}, Total Cars: {self.totalCars}, "
                f"Cars per Lane: {self.carsPerLane}, Green Time: {self.greenTime}s")
       # totalCars = 20  # Simulated number of cars

class Intersection:
    def __init__(self, intersectionID, trafficLights):
        self.intersectionID = intersectionID
        self.trafficLights = trafficLights
    def getIntersectionID(self):
        return self.intersectionID
    def getTrafficLights(self):
        return self.trafficLights
    def getSignalOn(self):
        for light in self.trafficLights:
            if light.greenTime > 0:
                return light.getTrafficID
        return None
    def getTrafficLightwithID(self, trafficID):
        for light in self.trafficLights:
            if light.getTrafficID() == trafficID:
                return light
        return None


class CircularTrafficLights:
    def __init__(self, intersection):
        self.lights = intersection.getTrafficLights()
        self.intersection = intersection
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if not self.lights:
            raise StopIteration
        light = self.lights[self.index]
        self.index = (self.index + 1) % len(self.lights)
        return light
    def getIntersection(self):
        return self.intersection


    # def update_light(self, trafficID, totalCars, carsPerLane, greenTime):
    #     for light in self.lights:
    #         if light.trafficID == trafficID:
    #             light.update_traffic(totalCars, carsPerLane, greenTime)
    #             print(f"Updated Traffic Light {trafficID}")

def save_to_database(data, filename='testA.json'):
    try:
        with open(filename, 'r') as json_file:
            existing_data = json.load(json_file)
            if not isinstance(existing_data, list):
                existing_data = []
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    
    data_dict = {
        f"Cycle {data[0]}": {
            "trafficID": data[1],
            "num_cars": data[2],
            "weatherStamp:": data[3],
            "timeStamp": data[4],
            "dateStamp": data[5],
            "greenTime": data[6]
        }
    }

    existing_data.append(data_dict)  # Use append to add the dictionary to the list

    with open(filename, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

In [4]:


intersection1 = [
    TrafficLight(1, 4, True), # trafficID, lanes
    TrafficLight(2, 4, True),
    TrafficLight(3, 4, False),
    TrafficLight(4, 4, False)
]
intersection2 = [
    TrafficLight(1, 4,True), # trafficID, lanes
    TrafficLight(2, 4,False),
    TrafficLight(3, 4, False),
    TrafficLight(4, 4, True)
]
intersection3 = [
    TrafficLight(1, 4, False), # trafficID, lanes
    TrafficLight(2, 4, True),
    TrafficLight(3, 4, True),
    TrafficLight(4, 4, False)
]
intersection4 = [
    TrafficLight(1, 4, False), # trafficID, lanes
    TrafficLight(2, 4, False),
    TrafficLight(3, 4, True),
    TrafficLight(4, 4, True)
]
network = [Intersection('A',intersection1), Intersection('B',intersection2), Intersection('C',intersection3), Intersection('D',intersection4)]
# traffic_circle = []
# for i, intersection in enumerate(network):
#     traffic_circle.append(CircularTrafficLights(intersection))

traffic_circle1 = CircularTrafficLights(network[0])
traffic_circle2 = CircularTrafficLights(network[1])
traffic_circle3 = CircularTrafficLights(network[2])
traffic_circle4 = CircularTrafficLights(network[3])

In [5]:
def setup_delayed_task(func, delay, *args):
    timer = threading.Timer(delay, func, args=args)
    timer.start()
    return
def processTraffic(circularTrafficLights):
    i = 0
    cycle = 0
    delay_seconds = 30 
    for light in circularTrafficLights:
        if(i % len(circularTrafficLights.getIntersection().getTrafficLights()) == 0):
            cycle += 1
        if(cycle == 1000):
            break
        i += 1
        time.sleep(2)
        numOfCars = light.getTotalCars()
        if light.generateCars:
            numOfCars = np.random.randint(0, 120)  # Simulate number of cars
            light.update_traffic(numOfCars)
        # while True:
        #     try:
        #         history, numOfCars = generate_estimation()
        #         numOfCars = np.random.randint(0, 120)  # Simulate number of cars
        #         break  # Exit the loop if the function succeeds
        #     except Exception as e:
        #         print(f"Error occurred: {e}. Retrying...")

        weatherStamp = 33
        timeStamp = datetime.now().strftime("%H:%M:%S")
        dateStamp = datetime.now().strftime("%Y-%m-%d")

        data = [
            cycle, light.trafficID, numOfCars,
            weatherStamp, timeStamp, dateStamp,
            light.getGreenTime()
        ]
        save_to_database(data, f"intersection_{circularTrafficLights.getIntersection().getIntersectionID()}.json")
        print(light)
        time.sleep(light.getGreenTime())
        if circularTrafficLights.getIntersection().getIntersectionID() == "A":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
        elif circularTrafficLights.getIntersection().getIntersectionID() == "B":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
        elif circularTrafficLights.getIntersection().getIntersectionID() == "C":
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
        elif circularTrafficLights.getIntersection().getIntersectionID() == "D":
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getGreenTime(), light.lanes)
        light.resetTotalCars()
        light.resetGreenTime()


def L1(intersection,trafficLightID, greenTime, lanes):
    totalCars =floor(max(0, (greenTime*(lanes + 1))//2.5)) 
    if(intersection.getIntersectionID() == "A"):
        if(trafficLightID == 1):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 2):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "B"):
        if(trafficLightID == 1):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))

def L2(intersection,trafficLightID, greenTime, lanes):
    totalCars = floor(max(0, (greenTime*(lanes + 1))//2.5))
    if(intersection.getIntersectionID() == "A"):
        if(trafficLightID == 1):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 2):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "C"):
        if(trafficLightID == 2):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 3):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.8 * totalCars))

def L3(intersection,trafficLightID, greenTime, lanes):
    totalCars = floor(max(0, (greenTime*(lanes + 1))//2.5))
    if(intersection.getIntersectionID() == "C"):
        if(trafficLightID == 1):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 2):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "D"):
        if(trafficLightID == 1):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))

def L4(intersection,trafficLightID, greenTime, lanes):
    totalCars = floor(max(0, (greenTime*(lanes + 1))//2.5))
    if(intersection.getIntersectionID() == "B"):
        if(trafficLightID == 2):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 1):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 4):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "D"):
        if(trafficLightID == 2):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 3):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 4):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))

In [6]:
threadA = threading.Thread(target=processTraffic, args=(traffic_circle1,))
threadB = threading.Thread(target=processTraffic, args=(traffic_circle2,))
threadC = threading.Thread(target=processTraffic, args=(traffic_circle3,))
threadD = threading.Thread(target=processTraffic, args=(traffic_circle4,))

threadA.start()
threadB.start()
threadC.start()
threadD.start()

threadA.join()
threadB.join()
threadC.join()
threadD.join()


Traffic Light 1 -> Lanes: 4, Total Cars: 100, Cars per Lane: 25, Green Time: 50s
Traffic Light 1 -> Lanes: 4, Total Cars: 0, Cars per Lane: 0, Green Time: 3s
Traffic Light 1 -> Lanes: 4, Total Cars: 0, Cars per Lane: 0, Green Time: 3s
Traffic Light 1 -> Lanes: 4, Total Cars: 55, Cars per Lane: 14, Green Time: 28s
Traffic Light 2 -> Lanes: 4, Total Cars: 102, Cars per Lane: 26, Green Time: 50sTraffic Light 2 -> Lanes: 4, Total Cars: 0, Cars per Lane: 0, Green Time: 3s

Traffic Light 3 -> Lanes: 4, Total Cars: 107, Cars per Lane: 27, Green Time: 50s


In [ ]:
# cnt = 0
# for light in traffic_circle1:
#     print(light.)
#     cnt+=1
#     if cnt == 4:
#         break

In [ ]:
def update_traffic_ids(file_path, intersection_id):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for cycle in data:
        for cycle_name, cycle_data in cycle.items():
            traffic_id = cycle_data['trafficID']
            cycle_data['trafficID'] = f"{intersection_id}{traffic_id}"
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# File path for intersection_A.json
for intersection in network:
    file_path = f'intersection_{intersection.getIntersectionID()}.json'
    update_traffic_ids(file_path, intersection.getIntersectionID())


print("Traffic IDs updated successfully.")


Traffic IDs updated successfully.


In [ ]:
import glob

# Function to load and merge JSON files
def load_and_merge_json(file_paths):
    merged_data = []
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            data = json.load(f)
            for entry in data:
                for cycle_data in entry.values():
                    merged_data.append(cycle_data)
    return merged_data

# Function to sort data based on dateStamp and timeStamp
def sort_data(data):
    sorted_data = sorted(data, key=lambda x: (x['dateStamp'], x['timeStamp']))
    return sorted_data

# List of JSON files
file_paths = ['intersection_A.json', 'intersection_B.json', 'intersection_C.json', 'intersection_D.json']

# Load and merge data
merged_data = load_and_merge_json(file_paths)

# Sort the merged data
sorted_data = sort_data(merged_data)

# Save the sorted data to a new JSON file
output_file = 'merged_sorted_intersections.json'
with open(output_file, 'w') as f:
    json.dump(sorted_data, f, indent=4)

print(f"Merged and sorted data saved to {output_file}")


Merged and sorted data saved to merged_sorted_intersections.json


In [ ]:
import json
import csv

# Load the merged and sorted JSON data
input_file = 'merged_sorted_intersections.json'
with open(input_file, 'r') as f:
    data = json.load(f)

# Define the output CSV file
output_file = 'merged_sorted_intersections.csv'

# Define the CSV columns
columns = ['trafficID', 'num_cars', 'weatherStamp:', 'timeStamp', 'dateStamp', 'greenTime']

# Write data to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    for entry in data:
        writer.writerow(entry)

print(f"Data has been written to {output_file}")


Data has been written to merged_sorted_intersections.csv


In [ ]:
import pandas as pd
#oopen csv file and display first 10 raws
df = pd.read_json('merged_sorted_intersections.json')
print(df.head(10))


  trafficID  num_cars  weatherStamp:           timeStamp   dateStamp  \
0        A1         4             33 2024-08-01 23:59:10  2024-07-31   
1        B1        45             33 2024-08-01 23:59:10  2024-07-31   
2        C1         0             33 2024-08-01 23:59:10  2024-07-31   
3        D1         0             33 2024-08-01 23:59:10  2024-07-31   
4        A2        71             33 2024-08-01 23:59:15  2024-07-31   
5        C2        93             33 2024-08-01 23:59:15  2024-07-31   
6        D2         0             33 2024-08-01 23:59:15  2024-07-31   
7        D3        94             33 2024-08-01 23:59:20  2024-07-31   
8        B2         0             33 2024-08-01 23:59:35  2024-07-31   
9        B3         0             33 2024-08-01 23:59:40  2024-07-31   

   greenTime  
0          3  
1         23  
2          3  
3          3  
4         36  
5         47  
6          3  
7         47  
8          3  
9          3  


In [ ]:
len(df)

1079